In [28]:
import os
import gc
import sys
import argparse
import time
import numpy as np
import matplotlib.pyplot as plt

import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import utils, transforms, datasets
from torchvision.models.detection.rpn import AnchorGenerator

from transforms import ToTensor, Resize, Compose, testTensor
from data.kitti_dataset import KITTI
from backbone.backbone_vgg import vgg16, vgg11, vgg13, vgg19
from backbone.backbone_resnet import resnet101, resnet34
from models.faster_rcnn_mod import FasterRCNN
from models.mask_rcnn_mod import MaskRCNN
from models.keypoint_rcnn_mod import KeypointRCNN
from models.retinanet_mod import RetinaNet

from ID.intrinsic_dimension import estimate, block_analysis
from scipy.spatial.distance import pdist,squareform
from collections import OrderedDict
from tqdm import tqdm
from PIL import Image

In [2]:
resnet_net = torchvision.models.resnet101(pretrained=True)
modules = list(resnet_net.children())[:-1]
backbone = nn.Sequential(*modules)
backbone.out_channels = 2048
anchor_generator = AnchorGenerator(sizes=((32, 64, 128, 256, 512),), aspect_ratios=((0.5,1.0,2.0),)) 
roi_pooler = torchvision.ops.MultiScaleRoIAlign(featmap_names=['0'], output_size=7, sampling_ratio=2)

Downloading: "https://download.pytorch.org/models/resnet101-5d3b4d8f.pth" to C:\Users\ajayc/.cache\torch\hub\checkpoints\resnet101-5d3b4d8f.pth


  0%|          | 0.00/170M [00:00<?, ?B/s]

In [3]:
root = os.getcwd()
model_path = os.path.join(root, "trained_model", "coco", "faster_rcnn", "resnet_101")

In [4]:
model = FasterRCNN(backbone, num_classes=91,rpn_anchor_generator= anchor_generator, box_roi_pool=roi_pooler )
#model = RetinaNet(backbone, num_classes=10, anchor_generator=anchor_generator)
model.load_state_dict(torch.load(os.path.join(model_path, 'model.pt'), map_location=torch.device('cpu')))

<All keys matched successfully>

In [5]:
model.eval()

FasterRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (4): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (

In [6]:
class test_data(object):
    def __init__(self, path, transforms):
        self.path = path
        self.transforms = transforms
        self.imgs = list(sorted(os.listdir(path)))
       
    
    def __getitem__(self, idx):
        print(idx)
        img_path = os.path.join(self.path, self.imgs[idx])
        
        image = Image.open(img_path).convert('RGB')
        if self.transforms is not None:
            image = self.transforms(image)
        return image
    
    def __len__(self):
        return len(self.imgs)

In [10]:
image_directory = 'D:/Dataset/COCO/horizontal_shift'

data = test_data(image_directory, transforms= testTensor())
indices = torch.randperm(len(data))
dataset = torch.utils.data.Subset(data, indices[:24])

def collate_fn(batch):
    return tuple(zip(*batch))
    

In [38]:
testdata = DataLoader(dataset, batch_size=1, shuffle=True)

In [83]:
image = next(iter(testdata))

tensor(4937)


In [84]:
def get_boxes(model, image):
    pred = model(image)
    boxes = [i["boxes"] for i in pred]
    return boxes

In [85]:
def get_featmap_and_size(model, image):
    out0 = image
    out1 = model.backbone[3](F.relu(model.backbone[1](model.backbone[0](out0))))
    out2 = model.backbone[4](out1)
    out3 = model.backbone[5](out2)
    out4 = model.backbone[6](out3)
    out5 = model.backbone[8](model.backbone[7](out4))
    x = OrderedDict()
    x['0'] = out5
    x['1'] = out4
    image_size = [300,300]
    del out0,out1,out2,out3,out4
    return x,image_size

In [86]:
out0 = image
print(out0.shape)
out1 = model.backbone[3](F.relu(model.backbone[1](model.backbone[0](out0))))
print(out1.shape)
out2 = model.backbone[4](out1)
print(out2.shape)
out3 = model.backbone[5](out2)
print(out3.shape)
out4 = model.backbone[6](out3)
print(out4.shape)
out5 = model.backbone[8](model.backbone[7](out4))
print(out5.shape)
out6 = model.rpn.head.cls_logits(F.relu(model.rpn.head.conv(out5)))
print(out6.shape)
out7 = model.rpn.head.bbox_pred(F.relu(model.rpn.head.conv(out5)))
print(out7.shape)

torch.Size([1, 3, 300, 300])
torch.Size([1, 64, 75, 75])
torch.Size([1, 256, 75, 75])
torch.Size([1, 512, 38, 38])
torch.Size([1, 1024, 19, 19])
torch.Size([1, 2048, 1, 1])
torch.Size([1, 15, 1, 1])
torch.Size([1, 60, 1, 1])


In [87]:
boxes = get_boxes(model, image)

In [88]:
x, image_size = get_featmap_and_size(model, image)

In [89]:
out8 = model.roi_heads.box_roi_pool(x,boxes,image_size).flatten(start_dim=1)

In [90]:
print(out8.shape)
out8 = out8[0, :]
print(out8.shape)

torch.Size([13, 100352])
torch.Size([100352])


In [91]:
out9 = F.relu(model.roi_heads.box_head.fc7(F.relu(model.roi_heads.box_head.fc6(out8))))
print(out9.shape)

torch.Size([1024])


In [92]:
out10 = model.roi_heads.box_predictor.cls_score(out9)
out11 = model.roi_heads.box_predictor.bbox_pred(out9)

In [49]:
def computeID(r, nres, fraction):
    ID = []
    n = int(np.round(r.shape[0]*fraction))
    print(n)
    dist = squareform(pdist(r, 'euclidean'))
    print(dist)
    for i in range(nres):
        dist_s = dist
        perm = np.random.permutation(dist.shape[0])[0:n]
        print(perm)
        dist_s = dist_s[perm,:]
        dist_s = dist_s[:,perm]
        print(dist_s)
        ID.append(estimate(dist_s)[2])
    mean = np.mean(ID)
    error = np.std(ID)
    return mean,error

In [50]:
Out0 = out0.view(image.shape[0], -1).cpu().data
Out1 = out1.view(image.shape[0], -1).cpu().data
Out2 = out2.view(image.shape[0], -1).cpu().data
Out3 = out3.view(image.shape[0], -1).cpu().data
Out4 = out4.view(image.shape[0], -1).cpu().data
Out5 = out5.view(image.shape[0], -1).cpu().data
Out6 = out6.view(image.shape[0], -1).cpu().data
Out7 = out7.view(image.shape[0], -1).cpu().data
Out8 = out8.view(image.shape[0], -1).cpu().data
Out9 = out9.view(image.shape[0], -1).cpu().data
Out10 = out10.view(image.shape[0], -1).cpu().data
Out11 = out11.view(image.shape[0], -1).cpu().data

In [93]:
Out0 = torch.cat((Out0, out0.view(image.shape[0], -1).cpu().data),0)
Out1 = torch.cat((Out1, out1.view(image.shape[0], -1).cpu().data),0) 
Out2 = torch.cat((Out2, out2.view(image.shape[0], -1).cpu().data),0) 
Out3 = torch.cat((Out3, out3.view(image.shape[0], -1).cpu().data),0)                 
Out4 = torch.cat((Out4, out4.view(image.shape[0], -1).cpu().data),0) 
Out5 = torch.cat((Out5, out5.view(image.shape[0], -1).cpu().data),0) 
Out6 = torch.cat((Out6, out6.view(image.shape[0], -1).cpu().data),0) 
Out7 = torch.cat((Out7, out7.view(image.shape[0], -1).cpu().data),0)                 
Out8 = torch.cat((Out8, out8.view(image.shape[0], -1).cpu().data),0)
Out9 = torch.cat((Out9, out9.view(image.shape[0], -1).cpu().data),0) 
Out10 = torch.cat((Out10, out10.view(image.shape[0], -1).cpu().data),0)                 
Out11 = torch.cat((Out11, out11.view(image.shape[0], -1).cpu().data),0)

In [94]:
out = [Out0, Out1, Out2, Out3, Out4, Out5, Out6, Out7, Out8, Out9]

In [95]:
ID_all = []
for j in tqdm(range(0,12)):
    r = out[j]
    print(r)
    ID_all.append(computeID(r, 20, 0.9))
ID_all = np.array(ID_all)

 25%|██▌       | 3/12 [00:00<00:00, 19.36it/s]

tensor([[1.0000, 1.0000, 1.0000,  ..., 0.3216, 0.3294, 0.3255],
        [0.2314, 0.2118, 0.2196,  ..., 0.5725, 0.5176, 0.5137],
        [0.5333, 0.5176, 0.4980,  ..., 0.2392, 0.2314, 0.2392],
        [0.1765, 0.1647, 0.1490,  ..., 0.3490, 0.3843, 0.6000]])
4
[[  0.         233.40127477 196.38162408 192.24529053]
 [233.40127477   0.         172.60340044 179.01064801]
 [196.38162408 172.60340044   0.         174.79097167]
 [192.24529053 179.01064801 174.79097167   0.        ]]
[0 1 3 2]
[[  0.         233.40127477 192.24529053 196.38162408]
 [233.40127477   0.         179.01064801 172.60340044]
 [192.24529053 179.01064801   0.         174.79097167]
 [196.38162408 172.60340044 174.79097167   0.        ]]
[0 3 1 2]
[[  0.         192.24529053 233.40127477 196.38162408]
 [192.24529053   0.         179.01064801 174.79097167]
 [233.40127477 179.01064801   0.         172.60340044]
 [196.38162408 174.79097167 172.60340044   0.        ]]
[0 1 3 2]
[[  0.         233.40127477 192.24529053 196.381

 50%|█████     | 6/12 [00:00<00:00, 21.36it/s]

[[  0.         116.34057631 113.91685741 114.52799343]
 [116.34057631   0.         105.91260544 108.10166637]
 [113.91685741 105.91260544   0.         104.53578943]
 [114.52799343 108.10166637 104.53578943   0.        ]]
[3 0 1 2]
[[  0.         114.52799343 116.34057631 113.91685741]
 [114.52799343   0.         108.10166637 104.53578943]
 [116.34057631 108.10166637   0.         105.91260544]
 [113.91685741 104.53578943 105.91260544   0.        ]]
[0 1 2 3]
[[  0.         108.10166637 104.53578943 114.52799343]
 [108.10166637   0.         105.91260544 116.34057631]
 [104.53578943 105.91260544   0.         113.91685741]
 [114.52799343 116.34057631 113.91685741   0.        ]]
[1 0 2 3]
[[  0.         108.10166637 105.91260544 116.34057631]
 [108.10166637   0.         104.53578943 114.52799343]
 [105.91260544 104.53578943   0.         113.91685741]
 [116.34057631 114.52799343 113.91685741   0.        ]]
[0 1 2 3]
[[  0.         108.10166637 104.53578943 114.52799343]
 [108.10166637   0.  

 83%|████████▎ | 10/12 [00:00<00:00, 22.92it/s]


[2 1 0 3]
[[  0.         117.1766878  116.55802504 109.8298481 ]
 [117.1766878    0.          19.9542422   21.42726417]
 [116.55802504  19.9542422    0.          15.42948974]
 [109.8298481   21.42726417  15.42948974   0.        ]]
[3 1 2 0]
[[  0.          21.42726417 109.8298481   15.42948974]
 [ 21.42726417   0.         117.1766878   19.9542422 ]
 [109.8298481  117.1766878    0.         116.55802504]
 [ 15.42948974  19.9542422  116.55802504   0.        ]]
[0 1 2 3]
[[  0.          19.9542422  116.55802504  15.42948974]
 [ 19.9542422    0.         117.1766878   21.42726417]
 [116.55802504 117.1766878    0.         109.8298481 ]
 [ 15.42948974  21.42726417 109.8298481    0.        ]]
[0 2 3 1]
[[  0.         116.55802504  15.42948974  19.9542422 ]
 [116.55802504   0.         109.8298481  117.1766878 ]
 [ 15.42948974 109.8298481    0.          21.42726417]
 [ 19.9542422  117.1766878   21.42726417   0.        ]]
tensor([[0.0000, 0.0000, 0.0000,  ..., 0.1403, 0.0000, 0.0000],
        [0.

IndexError: list index out of range

In [96]:
ID_all

[(30.041128588837562, 7.105427357601002e-15),
 (21.955625330611785, 0.0),
 (45.2505498726059, 7.105427357601002e-15),
 (53.08736931843189, 0.0),
 (1016.1792288730236, 2.2737367544323206e-13),
 (8.854431614525343, 1.7763568394002505e-15),
 (2.039571260666321, 0.0),
 (4.033114941388432, 0.0),
 (7.7223762846517445, 0.0),
 (3.2016714497934857, 0.0)]